# Movie Industry Analysis

Authors: Cayke Felipe dos Anjos, Dolgor Purbueva

<img src="images/screen.png" alt="Alt text" width="1200"/>

## Overview  
This project analyzes past movies data to offer strategic business recommendations for a new film studio. We aim to predict the most profitable strategies for film production and release by examining production budgets, gross revenues, net profits, genres, popularity, key staff and release timelines. As result this project provides three business recommendations: what genre should a future movie be, what budget to allocate and when to release it.

## Business Problem

The company is expanding its portfolio by investing in a new film studio. Launching a film studio in today's competitive entertainment industry requires a solid understanding of what drives movie success and attracts audiences. The movie industry is known for its substantial risks and high capital demands. Recent developments in AI have made video content creation faster and more efficient, increasing competition but also opening up new opportunities to enter the market.

Our project aims to analyze various datasets, including past movies' financial results, genre correlations, and movie ratings and popularity. By using data analysis techniques, we seek to gain valuable insights and identify patterns that can help shape the company's film production strategy. The goal is to provide three concrete business recommendations that maximize profitability and lower business risks, ensuring a strong entry into the market.

Questions we tried to answer with analysis:
* How should the movie be budgeted?
* What genres are most profitable?
* When should a movie be released?

## Data Understanding

We used datasets from Rotten Tomatoes, TheMovieDB, Bom Office Mojo, IMDB and The Numbers. Each dataset is of different size and contains different data catecories which might be seen as a limitation. For every question we answer we choose the most relevant dataset or merge some of them together for a fuller picture.

For budget related analysis we used IMDB and The Numbers. TheMovieDD dataset was used to explore movie ratings and popularity.

* One of the main metrics we explored is genre. [IMDB defines](https://help.imdb.com/article/contribution/titles/genres/GZDRMS6R742JRGAG?ref_=helpms_h#) genre as a category of artistic composition, characterized by similarities in form, style, or subject matter for a piece of content. [Reaserch by Mustafa Mahmoud Yousry](https://offscreen.com/view/going_to_the_movies) has shown that genre is the main decision factor for audience when chosing a movie to watch.

<img src="images/genre.png" alt="Alt text" width="400"/>

* We introduced the Return on Investment (ROI) metric as a standardized criterion to measure the financial success of a movie relative to its investments, as well as calculated movie net profit.

* We calculated Net Profit.

<img src="images/movie_data_erd.jpeg" alt="Alt text" width="1200"/>

## Data Analysis

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from scipy import stats
import matplotlib.pyplot as plt
from math import ceil, floor
import seaborn as sns

## A risk analysis of the movie production business

In order to provide any business recommendation, we need to understand how much money it is being bet and how much profit is desired in entering this industry. 


Our concept of risk group is then defined: a high risk group is one with a large budget that, if successful, will yield large profit, but if failed, can result in a great loss.

We start by cleaning The Numbers dataset and trying to explain correlations between profit, loss and budget.

In [2]:
#function for cleaning columns and coverting to integer

def clean_and_convert_to_int(column):
    return column.replace({'\$': '', ',': ''}, regex=True).astype(int)

#function for converting to date type

def convert_to_date(column):
    return pd.to_datetime(column, errors='coerce')

#function to clean object columns

def clean_object(column):
    return column.strip().lower()

def split(column):
    return column.str.split(',')

In [4]:
#Cleaning The Numbers DF
tn = pd.read_csv('zippedData/tn.movie_budgets.csv')

#Converting strings to int values. Ex: '$1922819' -> 1922819
tn[['production_budget', 'domestic_gross', 'worldwide_gross']] = tn[['production_budget', 'domestic_gross', 'worldwide_gross']].apply(clean_and_convert_to_int)

#Getting release date in timestamp format to get the year
tn[['release_date']] = tn[['release_date']].apply(convert_to_date)
tn['release_year'] = tn['release_date'].dt.year

#id is an internal identifier
tn = tn.drop(columns=['id'])

#Calculating Net Profit. We only regards international success. A lot of movies have huge budgets and only become profitable internationally
tn['worldwide_net'] = tn['worldwide_gross'] - tn['production_budget']

#Calculating Return Over Investment
tn['ROI'] = (tn['worldwide_net']) / tn['production_budget']

tn = tn[(tn['domestic_gross'] != 0) & (tn['worldwide_gross'] != 0)]
tn = tn[(tn['ROI'] != 0) & (tn['ROI'] != np.inf) & (tn['ROI'] != -np.inf) & (tn['ROI'] != -100)]
tn = tn.sort_values(by='ROI')

tn

FileNotFoundError: [Errno 2] No such file or directory: 'zippedData/tn.movie_budgets.csv'

In [ ]:
tn_loss = tn[tn['worldwide_net']<0]
tn_profit = tn[tn['worldwide_net']>0]

In [ ]:
fig,[ax1,ax2] = plt.subplots(ncols=2,figsize=(10,6),gridspec_kw={'wspace':0.4})

p = sns.regplot(x=tn_loss['production_budget']/10**6,y=tn_loss['worldwide_net']/10**6,color='r',ax=ax1)
ax1.set_title('Correlation Between Budget and Loss')
ax1.set_xlabel('Budget (Millions of $)')
ax1.set_ylabel('Worldwide Loss')
ax1.set_ylim(-125,5)
slope, intercept, r, p_value1, sterr = stats.linregress(x=p.get_lines()[0].get_xdata(),
                                                        y=p.get_lines()[0].get_ydata())

#add regression equation to plot
ax1.text(0, -120, 'y = ' + str(round(slope,3)) + 'x'+  str(round(intercept,3)))


p = sns.regplot(x=tn_profit['production_budget']/10**6,y=tn_profit['worldwide_net']/10**6,color='b',ax=ax2)
ax2.set_ylim(0,2500)
ax2.set_title('Correlation Between Budget and Profit')
ax2.set_xlabel('Budget (Millions of $)')
ax2.set_ylabel('Worldwide Profit')
slope, intercept, r2, p_value2, sterr = stats.linregress(x=p.get_lines()[0].get_xdata(),
                                                         y=p.get_lines()[0].get_ydata())

#add regression equation to plot
ax2.text(0, 2400, 'y = ' + str(round(slope,3)) + 'x'+ ' + '+ str(round(intercept,3)));
print(r2);

Based on the determination that a higher budget does indeed correlate with higher profits and losses, we divide the dataset into 3 groups with same size based on their risk level:
- High risk invetsment, higher budget movies that will probably not have a very high ROI but have the chance of making large profit if audiences enjoy it;
- Low risk investment, a lower cost that can have a high ROI based on the lower investment;
- Medium risk investment, that is just inbetween.

We first filter the dataset starting from 2000 to onli pick up recent trends and calculate the 2 budget thresholds for each group.

In [ ]:
tn_since2000 = tn[pd.to_datetime(tn['release_date']).dt.year>=2000]
tn_since2000

In [ ]:
# We will work with 2 thresholds for our analysis
budget = tn_since2000['production_budget'].sort_values()
n_budget = len(budget)
threshold_budget_low = budget.iloc[int(n_budget/3)]
threshold_budget_high = budget.iloc[int(2*n_budget/3)]
print(f'The threshold in between low and medium risk is at ${threshold_budget_low/10**6:.02f} \
millions and in between medium and high is ${threshold_budget_high/10**6:0.2f} millions.')

In [ ]:
tn_low_risk  = tn_since2000[tn_since2000['production_budget']<=threshold_budget_low]
tn_medium_risk = tn_since2000[(tn_since2000['production_budget']>threshold_budget_low) & (tn_since2000['production_budget']<threshold_budget_high)]
tn_high_risk = tn_since2000[tn_since2000['production_budget']>=threshold_budget_high]

In [ ]:
fig, ax = plt.subplots(nrows=2, figsize=(12,6), gridspec_kw={'hspace': 0.5})
# fig.tight_layout()


median_profit_low = tn_low_risk['worldwide_net'].median()/10**6
median_profit_medium = tn_medium_risk['worldwide_net'].median()/10**6
median_profit_high = tn_high_risk['worldwide_net'].median()/10**6
print(median_profit_low, median_profit_medium, median_profit_high)

ax[0].hist(tn_low_risk['production_budget']/10**6, color='b',bins=20, label='Low risk productions')
ax[0].hist(tn_medium_risk['production_budget']/10**6, color='purple',bins=20, label='Medium risk productions')
ax[0].hist(tn_high_risk['production_budget']/10**6, color='r',bins=100, label='High risk productions');
ax[0].set_yticks(np.arange(0,201,25))
ax[0].set_ylim(0,200)
ax[0].set_xticks(np.arange(0,501,10))
ax[0].set_xlim(0,200)
ax[0].set_xlabel('Production Budget (Millions of US Dollars)')
ax[0].set_ylabel('Count')
ax[0].legend()
ax[0].set_title('Distribution of Movies Based on Budget')
ax[0].vlines(x=threshold_budget_low/10**6,ymin=0,ymax=200,color='black')
# ax[0].text(x=threshold_budget_low/10**6,y=175,s='Median budget')
ax[0].vlines(x=threshold_budget_high/10**6,ymin=0,ymax=200,color='black')
# ax[0].text(x=threshold_budget_high,y=175,s='Median budget')

ax[1].scatter(x=tn_low_risk['production_budget']/10**6, y=tn_low_risk['worldwide_net']/10**6,color='b',label='Low risk productions')
ax[1].scatter(x=tn_medium_risk['production_budget']/10**6, y=tn_medium_risk['worldwide_net']/10**6,color='purple',label='Medium risk productions')
ax[1].scatter(x=tn_high_risk['production_budget']/10**6, y=tn_high_risk['worldwide_net']/10**6, color='r', label='High risk productions')
ax[1].set_title('International Profit Versus Budget')
ax[1].set_xlabel('Production budget (Millions of US Dollars)')
ax[1].set_ylabel('International Profit (Millions of US Dollars)')
ax[1].set_yticks(np.arange(0,1000,25))
ax[1].set_ylim(0.1,10000)
ax[1].set_xticks(np.arange(0,501,50))
ax[1].set_xlim(1,500)
ax[1].vlines(x=threshold_budget_low/10**6,ymin=0,ymax=200,color='black')
ax[1].vlines(x=threshold_budget_high/10**6,ymin=0,ymax=200,color='black')
ax[1].hlines(xmin=0,xmax=500,y=median_profit_low,color='blue', linestyle='dashed')
ax[1].hlines(xmin=0,xmax=500,y=median_profit_medium,color='purple', linestyle='dashed')
ax[1].hlines(xmin=0,xmax=500,y=median_profit_high,color='red', linestyle='dashed')
ax[1].set_yscale('log')
ax[1].text(x=335,y=0.8,s='Low Risk Median Profit (~$0.6 Million)')
ax[1].text(x=335,y=20,s='Medium Risk Median Profit (~$16 Million)')
ax[1].text(x=335,y=100,s='High Hisk Median Profit (~$85 Million)')

# ax[1].text(x=21,y=2000,s='Median Profit for Low Risk Movie')

# Low Risk - Budget < $11.50 millions

The first thing we need to do is join the table with budgets and profit with the table that contains genres. Genres are in both IMDB and Movie DB meaning that in order to lose the least amount of information, we need to perform joins with different tables, concatenate and remove duplicates.

In [ ]:
tn_low_risk

In [ ]:
conn = sqlite3.connect('data/im.db')

imdb = pd.read_sql(
    """
    SELECT  mb.primary_title AS movie,
            mb.start_year, 
            mb.runtime_minutes, 
            mb.genres, 
            per.primary_name AS director
    FROM movie_basics AS mb
    JOIN movie_akas AS ma
    USING(movie_id)
    JOIN directors AS dir
    USING(movie_id)
    JOIN persons AS per
    USING(person_id)
    GROUP BY movie_id
    
    """
, conn
)
imdb

In [ ]:
genre_dict = {
    '28': 'Action',
    '12': 'Adventure',
    '16': 'Animation',
    '35': 'Comedy',
    '80': 'Crime',
    '99': 'Documentary',
    '18': 'Drama',
    '10751': 'Family',
    '14': 'Fantasy',
    '36': 'History',
    '27': 'Horror',
    '10402': 'Music',
    '9648': 'Mystery',
    '10749': 'Romance',
    '878': 'Science Fiction',
    '10770': 'TV Movie',
    '53': 'Thriller',
    '10752': 'War',
    '37': 'Western'
}

In [ ]:
movie_db = pd.read_csv('data/tmdb.movies.csv')
movie_db['genre_ids'] = movie_db['genre_ids'].replace({'\[': '', '\]': ''}, regex=True)
movie_db['genre_ids'] = movie_db['genre_ids'].str.split(',')
movie_db = movie_db.explode('genre_ids')
movie_db['genre_ids'] = movie_db['genre_ids'].apply(clean_object)
movie_db['genre_ids'] = movie_db['genre_ids'].map(genre_dict)
movie_db.drop(labels=['Unnamed: 0', 'id', 'original_language', 'popularity', 'release_date','vote_average', 'vote_count'],axis=1,inplace=True)
movie_db.rename(columns={'title':'movie','genre_ids':'genres'},inplace=True)
movie_db

In [ ]:
df_lowrisk_merged1 = pd.merge(tn_low_risk,movie_db,how='inner',on='movie')
df_lowrisk_merged1.drop_duplicates(subset=['release_date','genres','movie'], inplace=True)
df_lowrisk_merged1.drop('original_title', axis=1, inplace=True)
df_lowrisk_merged1.dropna(subset=['genres', 'worldwide_net', 'movie','production_budget'],inplace=True)
df_lowrisk_merged1

In [ ]:
df_lowrisk_merged2 = pd.merge(left=tn_low_risk, right=movie_db,how='inner',left_on='movie', right_on='original_title')
df_lowrisk_merged2.drop_duplicates(subset=['release_date','genres','movie_x'], inplace=True)
df_lowrisk_merged2.rename(columns={'movie_x':'movie'},inplace=True)
df_lowrisk_merged2.drop(['original_title', 'movie_y'],axis=1,inplace=True)
df_lowrisk_merged2.dropna(subset=['genres', 'worldwide_net', 'movie','production_budget'],inplace=True)
df_lowrisk_merged2

In [ ]:
df_lowrisk_merged3 = pd.merge(tn_low_risk,imdb,how='inner',on='movie')
df_lowrisk_merged3.drop_duplicates(subset=['release_date','movie'], inplace=True)
df_lowrisk_merged3.drop(['runtime_minutes', 'start_year','director'], axis=1, inplace=True)
df_lowrisk_merged3.dropna(subset=['genres', 'worldwide_net', 'movie','production_budget'],inplace=True)
df_lowrisk_merged3

We explode the lists with the genres so that movies might be duplicate buteach entry will have a diffrent genre it is part of.

In [ ]:
df_lowrisk_merged3['genres'] = df_lowrisk_merged3['genres'].str.split(',')
df_lowrisk_merged3 = df_lowrisk_merged3.explode('genres')
# df_lowrisk_merged3['genres'].value_counts()

In [ ]:
df_lowrisk_exploded = pd.concat([df_lowrisk_merged1,df_lowrisk_merged2,df_lowrisk_merged3])
df_lowrisk_exploded.drop_duplicates(subset=['release_date','movie','genres'], inplace=True)
df_lowrisk_exploded['genres'].value_counts()

Finally, we rename same genres written diffrently.

In [ ]:
df_lowrisk_exploded['genres'].replace(to_replace='Science Fiction', value='Sci-Fi',inplace=True)
df_lowrisk_exploded['genres'].value_counts()

## Low Risk - Genre

Finally, the dataframe has now all the information from genres and movies we need.

In [ ]:
df_lowrisk_exploded.groupby('genres')['ROI'].median().sort_values(ascending=False).reset_index().head()

In [ ]:
df_lowrisk_exploded.groupby('genres')['worldwide_net'].median().sort_values(ascending=False).reset_index().head()

Horror and Mystery have very similar ROI's and international profits. In order to determine which is more profitable, we use a t-test.

In [ ]:
mean_horror = df_lowrisk_exploded[df_lowrisk_exploded['genres']=='Horror']['worldwide_net'].mean()/10**6
std_horror = df_lowrisk_exploded[df_lowrisk_exploded['genres']=='Horror']['worldwide_net'].std()/10**6
n_horror = len(df_lowrisk_exploded[df_lowrisk_exploded['genres']=='Horror']['worldwide_net'])

t_horror_20 = (20-mean_horror)/(std_horror/n_horror**0.5)
t_horror_35 = (35-mean_horror)/(std_horror/n_horror**0.5)
t_horror_50 = (50-mean_horror)/(std_horror/n_horror**0.5)
print(f'T scores for profit of: \n$20 million = {t_horror_20}\n$35 million = {t_horror_35}\n$50 million = {t_horror_50}\n')

prob_horror_20 = 1-stats.t.cdf(t_horror_20,df=n_horror-1)
prob_horror_35 = 1-stats.t.cdf(t_horror_35,df=n_horror-1)
prob_horror_50 = 1-stats.t.cdf(t_horror_50,df=n_horror-1)
print(f'Probability for profit of: \n$20 million = {prob_horror_20}\n$35 million = {prob_horror_35}\n$50 million = {prob_horror_50}')

horror_avg_cost = df_lowrisk_exploded[df_lowrisk_exploded['genres'] == 'Horror']['production_budget'].mean()
print(f'Average cost of Horror movies below ${threshold_budget_low/10**6:0.2f} million are ${horror_avg_cost/10**6:.02f} millions')

In [ ]:
mean_mystery = df_lowrisk_exploded[df_lowrisk_exploded['genres']=='Mystery']['worldwide_net'].mean()/10**6
std_mystery = df_lowrisk_exploded[df_lowrisk_exploded['genres']=='Mystery']['worldwide_net'].std()/10**6
n_mystery = len(df_lowrisk_exploded[df_lowrisk_exploded['genres']=='Mystery']['worldwide_net'])

t_mystery_20 = (20-mean_mystery)/(std_mystery/n_mystery**0.5)
t_mystery_35 = (35-mean_mystery)/(std_mystery/n_mystery**0.5)
t_mystery_50 = (50-mean_mystery)/(std_mystery/n_mystery**0.5)
print(f'T scores for profit of: \n$20 million = {t_mystery_20}\n$35 million = {t_mystery_35}\n$50 million = {t_mystery_50}\n')

prob_mystery_20 = 1-stats.t.cdf(t_mystery_20,df=n_mystery-1)
prob_mystery_35 = 1-stats.t.cdf(t_mystery_35,df=n_mystery-1)
prob_mystery_50 = 1-stats.t.cdf(t_mystery_50,df=n_mystery-1)
print(f'Probability for profit f: \n$20 million = {prob_mystery_20}\n$35 million = {prob_mystery_35}\n$50 million = {prob_mystery_50}')

mystery_avg_cost = df_lowrisk_exploded[df_lowrisk_exploded['genres'] == 'Mystery']['production_budget'].mean()
print(f'Average cost of Mystery movies below ${threshold_budget_low/10**6:0.2f} million are ${mystery_avg_cost/10**6:.02f} millions')

### This means that Horror movies will cost about the same and will have a higher chance of getting more profit.

## Low Risk - Director

Knowing which genre is more profitable, we need to choose directors who succed in making profitable horror movies. For this, we do not restrict the detaframe to early releases.

In [ ]:
df_merged = pd.merge(tn,imdb,how='inner',on='movie')
df_merged.drop_duplicates(subset=['release_date','movie'], inplace=True)
df_merged['genres'] = df_merged['genres'].str.split(',')
df_merged = df_merged.explode(column='genres')
df_horror = df_merged[df_merged['genres']=='Horror']
df_horror = df_horror[df_horror['production_budget']<=threshold_budget_low]
df_horror.sort_values(by='worldwide_net',ascending=False)

In [ ]:
director_horror = df_horror.groupby('director')['worldwide_net'].mean().sort_values(ascending=False).reset_index()
director_horror.head(10)

In [ ]:
df_horror[df_horror['director'].isin(director_horror['director'].loc[:5])].sort_values('director')

### For the animation movie, we recommend the directors:
- Jordan Peele;
- John R. Leonetti.

## Low risk - Release Month

In [ ]:
df_horror['release_month'] = pd.to_datetime(df_horror['release_date'], format='%m').dt.month
df_horror.drop('release_date',axis=1,inplace=True)
df_horror.head()

In [ ]:
horror_month_release = df_horror.groupby('release_month')['movie'].count().reset_index()

fig,ax = plt.subplots(figsize=(10,5))

ax.bar(x='release_month',height='movie',data=horror_month_release);
ax.set_xticks(np.arange(1,13,1))
ax.set_yticks(np.arange(0,30,5))
ax.set_xlabel('Month')
ax.set_ylabel('Number of movies released');
# ax.vlines(x=2,ymin=0,ymax=18, color='r')
ax.vlines(x=4,ymin=0,ymax=25, color='r')
ax.vlines(x=10,ymin=0,ymax=25, color='r')
# ax.text(x=2.1,y=17, s='Spring Break')
ax.text(x=4.1,y=22.5, s='Easter')
ax.text(x=10.05,y=22.5, s='Halloween')
ax.set_title('Release Dates for Horror Movies');

Two options of release dates.
- April - Easter: a lot of Horror movies have religious subtext;
- November - Halloween.

We recommend releasing the movie during Halloween.

## Low risk - Runtime

In [ ]:
n_horror = len(df_horror['runtime_minutes'].dropna())
mean_horror = df_horror['runtime_minutes'].dropna().mean()
std_horror = df_horror['runtime_minutes'].dropna().std()
Tcrit = stats.t.ppf(0.975,n_horror-1)

runtime_horror_low = mean_horror - Tcrit*std_horror/n_horror**0.5
runtime_horror_up = mean_horror + Tcrit*std_horror/n_horror**0.5

print(f'Runtime should be around {round(runtime_horror_low)} to {ceil(runtime_horror_up)} minutes')


# Medium Risk

We now repeat the same steps with medium risk budgets.

In [ ]:
df_mediumrisk_merged1 = pd.merge(tn_medium_risk,movie_db,how='inner',on='movie')
df_mediumrisk_merged1.drop_duplicates(subset=['release_date','genres','movie'], inplace=True)
df_mediumrisk_merged1.drop('original_title', axis=1, inplace=True)
df_mediumrisk_merged1

In [ ]:
df_mediumrisk_merged2 = pd.merge(left=tn_medium_risk, right=movie_db,how='inner',left_on='movie', right_on='original_title')
df_mediumrisk_merged2.drop_duplicates(subset=['release_date','genres','movie_x'], inplace=True)
df_mediumrisk_merged2.rename(columns={'movie_x':'movie'},inplace=True)
df_mediumrisk_merged2.drop(['original_title', 'movie_y'],axis=1,inplace=True)
df_mediumrisk_merged2

In [ ]:
df_mediumrisk_merged3 = pd.merge(tn_medium_risk,imdb,how='inner',on='movie')
df_mediumrisk_merged3.drop_duplicates(subset=['release_date','movie'], inplace=True)
df_mediumrisk_merged3.drop(['runtime_minutes', 'start_year','director'], axis=1, inplace=True)
df_mediumrisk_merged3

In [ ]:
df_mediumrisk_merged3['genres'] = df_mediumrisk_merged3['genres'].str.split(',')
df_mediumrisk_merged3 = df_mediumrisk_merged3.explode('genres')
df_mediumrisk_merged3['genres'].value_counts()

In [ ]:
df_mediumrisk_exploded = pd.concat([df_mediumrisk_merged1, df_mediumrisk_merged2, df_mediumrisk_merged3])
df_mediumrisk_exploded.drop_duplicates(subset=['release_date','movie','genres'], inplace=True)
df_mediumrisk_exploded['genres'].value_counts()

In [ ]:
df_mediumrisk_exploded['genres'].replace(to_replace='Science Fiction', value='Sci-Fi',inplace=True)
df_mediumrisk_exploded['genres'].value_counts()

## Medium Risk - Genre

In [ ]:
df_mediumrisk_exploded.groupby('genres')['worldwide_net'].count().sort_values(ascending=False).reset_index()

In [ ]:
df_mediumrisk_exploded['genres'].replace(to_replace='Musical', value='Music')
df_mediumrisk_exploded = df_mediumrisk_exploded[df_mediumrisk_exploded['genres']!='TV Movie']

In [ ]:
df_mediumrisk_exploded.groupby('genres')['worldwide_net'].median().sort_values(ascending=False).reset_index().head(10)

In [ ]:
#Let us calculate the probability that an horror and horror movie will make profits
mean_horror = df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Horror']['worldwide_net'].mean()/10**6
std_horror = df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Horror']['worldwide_net'].std()/10**6
n_horror = len(df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Horror']['worldwide_net'])

t_horror_1 = (20-mean_horror)/(std_horror/n_horror**0.5)
t_horror_2 = (40-mean_horror)/(std_horror/n_horror**0.5)
t_horror_3 = (60-mean_horror)/(std_horror/n_horror**0.5)
t_horror_4 = (80-mean_horror)/(std_horror/n_horror**0.5)
print(f'T scores for profit of: \n$20 million = {t_horror_1}\n$40 million = {t_horror_2}\n$60 million = {t_horror_3}\n$80 million = {t_horror_4}\n')

prob_horror_1 = 1-stats.t.cdf(t_horror_1,df=n_horror-1)
prob_horror_2 = 1-stats.t.cdf(t_horror_2,df=n_horror-1)
prob_horror_3 = 1-stats.t.cdf(t_horror_3,df=n_horror-1)
prob_horror_4 = 1-stats.t.cdf(t_horror_4,df=n_horror-1)
print(f'Probability for profit f: \n$20 million = {prob_horror_1}\n$40 million = {prob_horror_2}\n$60 million = {prob_horror_3}\n$80 million = {prob_horror_4}\n')

horror_avg_cost = df_mediumrisk_exploded[df_mediumrisk_exploded['genres'] == 'Horror']['production_budget'].mean()
print(f'Average cost of horror movies above ${threshold_budget_low/10**6:0.02f} millions but below ${threshold_budget_high/10**6} \
millions are ${horror_avg_cost/10**6:.02f} millions')

In [ ]:
#Let us calculate the probability that an music and music movie will make profits
mean_music = df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Music']['worldwide_net'].mean()/10**6
std_music = df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Music']['worldwide_net'].std()/10**6
n_music = len(df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Music']['worldwide_net'])

t_music_1 = (20-mean_music)/(std_music/n_music**0.5)
t_music_2 = (40-mean_music)/(std_music/n_music**0.5)
t_music_3 = (60-mean_music)/(std_music/n_music**0.5)
t_music_4 = (80-mean_music)/(std_music/n_music**0.5)
print(f'T scores for profit of: \n$20 million = {t_music_1}\n$40 million = {t_music_2}\n$60 million = {t_music_3}\n$80 million = {t_music_4}\n')

prob_music_1 = 1-stats.t.cdf(t_music_1,df=n_music-1)
prob_music_2 = 1-stats.t.cdf(t_music_2,df=n_music-1)
prob_music_3 = 1-stats.t.cdf(t_music_3,df=n_music-1)
prob_music_4 = 1-stats.t.cdf(t_music_4,df=n_music-1)
print(f'Probability for profit f: \n$20 million = {prob_music_1}\n$40 million = {prob_music_2}\n$60 million = {prob_music_3}\n$80 million = {prob_music_4}\n')

music_avg_cost = df_mediumrisk_exploded[df_mediumrisk_exploded['genres'] == 'Music']['production_budget'].mean()
print(f'Average cost of Music movies above ${threshold_budget_low/10**6:0.02f} millions but below ${threshold_budget_high/10**6} \
millions are ${music_avg_cost/10**6:.02f} millions')

In [ ]:
#Let us calculate the probability that an comedy and comedy movie will make profits
mean_comedy = df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Comedy']['worldwide_net'].mean()/10**6
std_comedy = df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Comedy']['worldwide_net'].std()/10**6
n_comedy = len(df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Comedy']['worldwide_net'])

t_comedy_1 = (20-mean_comedy)/(std_comedy/n_comedy**0.5)
t_comedy_2 = (40-mean_comedy)/(std_comedy/n_comedy**0.5)
t_comedy_3 = (60-mean_comedy)/(std_comedy/n_comedy**0.5)
t_comedy_4 = (80-mean_comedy)/(std_comedy/n_comedy**0.5)
print(f'T scores for profit of: \n$20 million = {t_comedy_1}\n$40 million = {t_comedy_2}\n$60 million = {t_comedy_3}\n$80 million = {t_comedy_4}\n')

prob_comedy_1 = 1-stats.t.cdf(t_comedy_1,df=n_comedy-1)
prob_comedy_2 = 1-stats.t.cdf(t_comedy_2,df=n_comedy-1)
prob_comedy_3 = 1-stats.t.cdf(t_comedy_3,df=n_comedy-1)
prob_comedy_4 = 1-stats.t.cdf(t_comedy_4,df=n_comedy-1)
print(f'Probability for profit f: \n$20 million = {prob_comedy_1}\n$40 million = {prob_comedy_2}\n$60 million = {prob_comedy_3}\n$80 million = {prob_comedy_4}\n')

comedy_avg_cost = df_mediumrisk_exploded[df_mediumrisk_exploded['genres'] == 'Comedy']['production_budget'].mean()
print(f'Average cost of Comedy movies above ${threshold_budget_low/10**6:0.02f} millions but below ${threshold_budget_high/10**6} \
millions are ${comedy_avg_cost/10**6:.02f} millions')

In [ ]:
#Let us calculate the probability that an fantasy and fantasy movie will make profits
mean_fantasy = df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Fantasy']['worldwide_net'].mean()/10**6
std_fantasy = df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Fantasy']['worldwide_net'].std()/10**6
n_fantasy = len(df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Fantasy']['worldwide_net'])

t_fantasy_1 = (20-mean_fantasy)/(std_fantasy/n_fantasy**0.5)
t_fantasy_2 = (40-mean_fantasy)/(std_fantasy/n_fantasy**0.5)
t_fantasy_3 = (60-mean_fantasy)/(std_fantasy/n_fantasy**0.5)
t_fantasy_4 = (80-mean_fantasy)/(std_fantasy/n_fantasy**0.5)
print(f'T scores for profit of: \n$20 million = {t_fantasy_1}\n$40 million = {t_fantasy_2}\n$60 million = {t_fantasy_3}\n$80 million = {t_fantasy_4}\n')

prob_fantasy_1 = 1-stats.t.cdf(t_fantasy_1,df=n_fantasy-1)
prob_fantasy_2 = 1-stats.t.cdf(t_fantasy_2,df=n_fantasy-1)
prob_fantasy_3 = 1-stats.t.cdf(t_fantasy_3,df=n_fantasy-1)
prob_fantasy_4 = 1-stats.t.cdf(t_fantasy_4,df=n_fantasy-1)
print(f'Probability for profit f: \n$20 million = {prob_fantasy_1}\n$40 million = {prob_fantasy_2}\n$60 million = {prob_fantasy_3}\n$80 million = {prob_fantasy_4}\n')

fantasy_avg_cost = df_mediumrisk_exploded[df_mediumrisk_exploded['genres'] == 'Fantasy']['production_budget'].mean()
print(f'Average cost of Fantasy movies above ${threshold_budget_low/10**6:0.02f} millions but below ${threshold_budget_high/10**6} \
millions are ${fantasy_avg_cost/10**6:.02f} millions')

In [ ]:
#Let us calculate the probability that an adventure and adventure movie will make profits
mean_adventure = df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Adventure']['worldwide_net'].mean()/10**6
std_adventure = df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Adventure']['worldwide_net'].std()/10**6
n_adventure = len(df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Adventure']['worldwide_net'])

t_adventure_1 = (20-mean_adventure)/(std_adventure/n_adventure**0.5)
t_adventure_2 = (40-mean_adventure)/(std_adventure/n_adventure**0.5)
t_adventure_3 = (60-mean_adventure)/(std_adventure/n_adventure**0.5)
t_adventure_4 = (80-mean_adventure)/(std_adventure/n_adventure**0.5)
print(f'T scores for profit of: \n$20 million = {t_adventure_1}\n$40 million = {t_adventure_2}\n$60 million = {t_adventure_3}\n$80 million = {t_adventure_4}\n')

prob_adventure_1 = 1-stats.t.cdf(t_adventure_1,df=n_adventure-1)
prob_adventure_2 = 1-stats.t.cdf(t_adventure_2,df=n_adventure-1)
prob_adventure_3 = 1-stats.t.cdf(t_adventure_3,df=n_adventure-1)
prob_adventure_4 = 1-stats.t.cdf(t_adventure_4,df=n_adventure-1)
print(f'Probability for profit f: \n$20 million = {prob_adventure_1}\n$40 million = {prob_adventure_2}\n$60 million = {prob_adventure_3}\n$80 million = {prob_adventure_4}\n')

adventure_avg_cost = df_mediumrisk_exploded[df_mediumrisk_exploded['genres'] == 'Adventure']['production_budget'].mean()
print(f'Average cost of Adventure movies above ${threshold_budget_low/10**6:0.02f} millions but below ${threshold_budget_high/10**6} \
millions are ${adventure_avg_cost/10**6:.02f} millions')

In [ ]:
#Let us calculate the probability that an family and family movie will make profits
mean_family = df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Family']['worldwide_net'].mean()/10**6
std_family = df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Family']['worldwide_net'].std()/10**6
n_family = len(df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Family']['worldwide_net'])

t_family_1 = (20-mean_family)/(std_family/n_family**0.5)
t_family_2 = (40-mean_family)/(std_family/n_family**0.5)
t_family_3 = (60-mean_family)/(std_family/n_family**0.5)
t_family_4 = (80-mean_family)/(std_family/n_family**0.5)
print(f'T scores for profit of: \n$20 million = {t_family_1}\n$40 million = {t_family_2}\n$60 million = {t_family_3}\n$80 million = {t_family_4}\n')

prob_family_1 = 1-stats.t.cdf(t_family_1,df=n_family-1)
prob_family_2 = 1-stats.t.cdf(t_family_2,df=n_family-1)
prob_family_3 = 1-stats.t.cdf(t_family_3,df=n_family-1)
prob_family_4 = 1-stats.t.cdf(t_family_4,df=n_family-1)
print(f'Probability for profit f: \n$20 million = {prob_family_1}\n$40 million = {prob_family_2}\n$60 million = {prob_family_3}\n$80 million = {prob_family_4}\n')

family_avg_cost = df_mediumrisk_exploded[df_mediumrisk_exploded['genres'] == 'Family']['production_budget'].mean()
print(f'Average cost of Family movies above ${threshold_budget_low/10**6:0.02f} millions but below ${threshold_budget_high/10**6} \
millions are ${family_avg_cost/10**6:.02f} millions')

### Genre for a medium risk production will be Music

## Medium Risk - Director

In [ ]:
df_music_movies = df_mediumrisk_exploded[df_mediumrisk_exploded['genres']=='Music']
df_music_directors = pd.merge(df_music_movies,imdb,how='inner',on='movie')

In [ ]:
df_music_directors.drop_duplicates(subset=['movie','director'],inplace=True)
df_music_directors.drop(['release_date','production_budget','domestic_gross','worldwide_gross','release_year','ROI','genres_x','start_year', 'runtime_minutes','genres_y'],axis=1,inplace=True)
df_music_directors.sort_values('director').sort_values('worldwide_net',ascending=False).head()

In [ ]:
director_count = df_music_directors.groupby('director')['worldwide_net'].count().reset_index().sort_values('worldwide_net',ascending=False)#.iloc[:15]#['director'].iloc[:7]
director_count = director_count.rename(columns={'worldwide_net':'count'})
# director_count = (director_count[director_count['count']>2])
director_count

Unfortunately, we just have one movie ffor each director, which does not allow us to extrapolate much.

In [ ]:
df_music_directors.groupby('director').median('worldwide_net').sort_values('worldwide_net', ascending=False)

For lack of data we recommend the director Damien Chazelle, as his Music movie sold 60% more than the second place.

## Medium Risk - Release Date

In [ ]:
df_music_movies.head()

In [ ]:
df_music_movies['release_month'] = pd.to_datetime(df_music_movies['release_date'], format='%m').dt.month
music_release_month = df_music_movies.groupby('release_month').count().reset_index()[['release_month','movie']].rename(columns={'movie':'count'})
music_release_month

In [ ]:
fig,ax = plt.subplots(figsize=(10,6))

ax.bar(x='release_month',height='count', data=music_release_month)
ax.set_xticks(np.arange(1,13))
ax.set_yticks(np.arange(0,11))
ax.set_ylim(0,10)
ax.set_xlabel('Release Month')
ax.set_ylabel('Number of Movies Released')
ax.set_title('Number of Music movies released each month')
ax.vlines(x=8,ymin=0,ymax=40,color='r')
ax.text(x=8.1,y=9,s='End of Summer');

### For some reason Music movies are more popular in August so we recommend this month for release.

## Medium risk - Runtime

In [ ]:
df_music = pd.merge(df_music_movies,imdb,how='inner',on='movie')
df_music.drop_duplicates(subset=['movie'],inplace=True)

runtime_music_mean = df_music['runtime_minutes'].mean()
runtime_music_std = df_music['runtime_minutes'].std()
n_music = len(df_music['runtime_minutes'])
err_music = runtime_music_std/n_music**0.5

Tcrit = stats.t.ppf(0.975,n_music-1)

runtime_music_upper = runtime_music_mean + Tcrit*err_music
runtime_music_lower = runtime_music_mean - Tcrit*err_music

print(f'The duration of the Music movie should be in between {floor(runtime_music_lower)} and {ceil(runtime_music_upper)} minutes.')

# High risk

We now do the same with the higher roisk budget.

In [ ]:
tn_high_risk

In [ ]:
df_highrisk_merged1 = pd.merge(tn_high_risk,movie_db,how='inner',on='movie')
df_highrisk_merged1.drop_duplicates(subset=['release_date','genres','movie'], inplace=True)
df_highrisk_merged1.drop('original_title', axis=1, inplace=True)
df_highrisk_merged1

In [ ]:
df_highrisk_merged2 = pd.merge(left=tn_high_risk, right=movie_db,how='inner',left_on='movie', right_on='original_title')
df_highrisk_merged2.drop_duplicates(subset=['release_date','genres','movie_x'], inplace=True)
df_highrisk_merged2.rename(columns={'movie_x':'movie'},inplace=True)
df_highrisk_merged2.drop(['original_title', 'movie_y'],axis=1,inplace=True)
df_highrisk_merged2

In [ ]:
df_highrisk_merged3 = pd.merge(tn_high_risk,imdb,how='inner',on='movie')
df_highrisk_merged3.drop_duplicates(subset=['release_date','movie'], inplace=True)
df_highrisk_merged3.drop(['runtime_minutes', 'start_year','director'], axis=1, inplace=True)
df_highrisk_merged3

In [ ]:
df_highrisk_merged3['genres'] = df_highrisk_merged3['genres'].str.split(',')
df_highrisk_merged3 = df_highrisk_merged3.explode('genres')
# df_highrisk_merged3['genres'].value_counts()

In [ ]:
df_highrisk_exploded = pd.concat([df_highrisk_merged1, df_highrisk_merged2, df_highrisk_merged3])
df_highrisk_exploded.drop_duplicates(subset=['release_date','movie','genres'], inplace=True)
df_highrisk_exploded['genres'].value_counts()

In [ ]:
df_highrisk_exploded['genres'].replace(to_replace='Science Fiction', value='Sci-Fi',inplace=True)
df_highrisk_exploded['genres'].value_counts()

## High risk - Genre

In [ ]:
df_highrisk_exploded.groupby('genres')['ROI'].median().sort_values(ascending=False).reset_index().head()

In [ ]:
df_highrisk_exploded.groupby('genres')['worldwide_net'].median().sort_values(ascending=False).reset_index().head()

In [ ]:
#Let us calculate the probability that an Animation, Sci-Fi and Adventure movie will make profits
#Animation first
mean_animation = df_highrisk_exploded[df_highrisk_exploded['genres']=='Animation']['worldwide_net'].mean()/10**6
std_animation = df_highrisk_exploded[df_highrisk_exploded['genres']=='Animation']['worldwide_net'].std()/10**6
n_animation = len(df_highrisk_exploded[df_highrisk_exploded['genres']=='Animation']['worldwide_net'])

t_animation_100 = (100-mean_animation)/(std_animation/n_animation**0.5)
t_animation_200 = (200-mean_animation)/(std_animation/n_animation**0.5)
t_animation_300 = (300-mean_animation)/(std_animation/n_animation**0.5)
t_animation_350 = (350-mean_animation)/(std_animation/n_animation**0.5)
print(f'T scores for profit of: \n$100 million = {t_animation_100}\n$200 million = {t_animation_200}\n$300 million = {t_animation_300}\n$350 million = {t_animation_350}\n')

prob_animation_100 = 1-stats.t.cdf(t_animation_100,df=n_animation-1)
prob_animation_200 = 1-stats.t.cdf(t_animation_200,df=n_animation-1)
prob_animation_300 = 1-stats.t.cdf(t_animation_300,df=n_animation-1)
prob_animation_350 = 1-stats.t.cdf(t_animation_350,df=n_animation-1)
print(f'Probability for profit f: \n$100 million = {prob_animation_100}\n$200 million = {prob_animation_200}\n$300 million = {prob_animation_300}\n$350 million = {prob_animation_350}\n')

animation_avg_cost = df_highrisk_exploded[df_highrisk_exploded['genres'] == 'Animation']['production_budget'].mean()
print(f'Average cost of animation movies above $20 million are ${animation_avg_cost/10**6:.02f} millions')

In [ ]:
#Let us calculate the probability that a scifi movie will make profits
mean_scifi = df_highrisk_exploded[df_highrisk_exploded['genres']=='Sci-Fi']['worldwide_net'].mean()/10**6
std_scifi = df_highrisk_exploded[df_highrisk_exploded['genres']=='Sci-Fi']['worldwide_net'].std()/10**6
n_scifi = len(df_highrisk_exploded[df_highrisk_exploded['genres']=='Sci-Fi']['worldwide_net'])

t_scifi_100 = (100-mean_scifi)/(std_scifi/n_scifi**0.5)
t_scifi_200 = (200-mean_scifi)/(std_scifi/n_scifi**0.5)
t_scifi_300 = (300-mean_scifi)/(std_scifi/n_scifi**0.5)
t_scifi_350 = (350-mean_scifi)/(std_scifi/n_scifi**0.5)
print(f'T scores for profit of: \n$100 million = {t_scifi_100}\n$200 million = {t_scifi_200}\n$300 million = {t_scifi_300}\n$350 million = {t_scifi_350}\n')

prob_scifi_100 = 1-stats.t.cdf(t_scifi_100,df=n_scifi-1)
prob_scifi_200 = 1-stats.t.cdf(t_scifi_200,df=n_scifi-1)
prob_scifi_300 = 1-stats.t.cdf(t_scifi_300,df=n_scifi-1)
prob_scifi_350 = 1-stats.t.cdf(t_scifi_350,df=n_scifi-1)
print(f'Probability for profit f: \n$100 million = {prob_scifi_100}\n$200 million = {prob_scifi_200}\n$300 million = {prob_scifi_300}\n$350 million = {prob_scifi_350}\n')

scifi_avg_cost = df_highrisk_exploded[df_highrisk_exploded['genres'] == 'Sci-Fi']['production_budget'].mean()
print(f'Average cost of Sci-Fi movies above $20 million are ${scifi_avg_cost/10**6:.02f} millions')

In [ ]:
#Let us calculate the probability that a adventure movie will make profits
mean_adventure = df_highrisk_exploded[df_highrisk_exploded['genres']=='Adventure']['worldwide_net'].mean()/10**6
std_adventure = df_highrisk_exploded[df_highrisk_exploded['genres']=='Adventure']['worldwide_net'].std()/10**6
n_adventure = len(df_highrisk_exploded[df_highrisk_exploded['genres']=='Adventure']['worldwide_net'])

t_adventure_100 = (100-mean_adventure)/(std_adventure/n_adventure**0.5)
t_adventure_200 = (200-mean_adventure)/(std_adventure/n_adventure**0.5)
t_adventure_300 = (300-mean_adventure)/(std_adventure/n_adventure**0.5)
t_adventure_350 = (350-mean_adventure)/(std_adventure/n_adventure**0.5)
print(f'T scores for profit of: \n$100 million = {t_adventure_100}\n$200 million = {t_adventure_200}\n$300 million = {t_adventure_300}\n$350 million = {t_adventure_350}\n')

prob_adventure_100 = 1-stats.t.cdf(t_adventure_100,df=n_adventure-1)
prob_adventure_200 = 1-stats.t.cdf(t_adventure_200,df=n_adventure-1)
prob_adventure_300 = 1-stats.t.cdf(t_adventure_300,df=n_adventure-1)
prob_adventure_350 = 1-stats.t.cdf(t_adventure_350,df=n_adventure-1)
print(f'Probability for profit f: \n$100 million = {prob_adventure_100}\n$200 million = {prob_adventure_200}\n$300 million = {prob_adventure_300}\n$350 million = {prob_adventure_350}\n')

adventure_avg_cost = df_highrisk_exploded[df_highrisk_exploded['genres'] == 'Adventure']['production_budget'].mean()
print(f'Average cost of Adventure movies above $20 million are ${adventure_avg_cost/10**6:.02f} millions')

## High Risk - Directors

FInally, for high risk budget.

In [ ]:
df_scifi_movies = df_highrisk_exploded[df_highrisk_exploded['genres']=='Sci-Fi']
df_scifi_directors = pd.merge(df_scifi_movies,imdb,how='inner',on='movie')
df_scifi_directors

In [ ]:
df_scifi_directors.drop_duplicates(subset=['movie','director'],inplace=True)
df_scifi_directors.drop(['release_date','production_budget','domestic_gross','worldwide_gross','release_year','ROI','genres_x','start_year', 'runtime_minutes','genres_y'],axis=1,inplace=True)
df_scifi_directors.sort_values('director')

In [ ]:
director_count = df_scifi_directors.groupby('director')['worldwide_net'].count().reset_index().sort_values('worldwide_net',ascending=False)#.iloc[:15]#['director'].iloc[:7]
director_count = director_count.rename(columns={'worldwide_net':'count'})
director_count = (director_count[director_count['count']>2])
director_count

In [ ]:
director_list = list(director_count['director'])
director_list

In [ ]:
scifi_topdirectors = df_scifi_directors[df_scifi_directors['director'].isin(director_list)].groupby('director')['worldwide_net'].median().reset_index().sort_values('worldwide_net',ascending=False)
scifi_topdirectors.rename(columns={'worldwide_net': 'Median Profit'})
#list_scifi_topdirectors

In [ ]:
df_scifi_directors[df_scifi_directors['director'].isin(scifi_topdirectors['director'])].sort_values('director')#.drop_duplicates(subset=['movie', 'director'])

Our recommendation for top directors are (in order of profit):
- For movies about robots, Michael Bay;
- For movies based on comics, Anthony Russo;
- For movies based on books, Francis Lawrence;

## High Risk - Release Month

In [ ]:
df_scifi_movies = df_highrisk_exploded[df_highrisk_exploded['genres']=='Sci-Fi']
df_scifi_movies

In [ ]:
df_scifi_movies['release_month'] = pd.to_datetime(df_scifi_movies['release_date'], format='%m').dt.month
df_scifi_movies.head()

In [ ]:
month_count = df_scifi_movies.groupby('release_month').count()['genres'].reset_index().rename(columns={'genres':'count'})

fig,ax = plt.subplots(figsize=(10,6))

ax.bar(x='release_month',height='count', data=month_count)
ax.set_xticks(np.arange(1,13))
ax.set_yticks(np.arange(0,45,5))
ax.set_ylim(0,40)
ax.set_xlabel('Release Month')
ax.set_ylabel('Number of Movies Released')
ax.set_title('Number of Sci-Fi movies released each month')
ax.vlines(x=3,ymin=0,ymax=40,color='r')
ax.vlines(x=6,ymin=0,ymax=40,color='r')
ax.text(x=3.1,y=38,s='Spring Break')
ax.text(x=6.1,y=38,s='Summer');

Movie should be released either in March for Spring Break or in June for Summer.

## High Risk - Runtime

In [ ]:
df_scifi = pd.merge(df_scifi_movies,imdb,how='inner',on='movie')
df_scifi.drop_duplicates(subset=['movie'],inplace=True)

runtime_scifi_mean = df_scifi['runtime_minutes'].mean()
runtime_scifi_std = df_scifi['runtime_minutes'].std()
n_scifi = len(df_scifi['runtime_minutes'])
err_scifi = runtime_scifi_std/n_scifi**0.5

Tcrit = stats.t.ppf(0.975,n_scifi-1)

runtime_scifi_upper = runtime_scifi_mean + Tcrit*err_scifi
runtime_scifi_lower = runtime_scifi_mean - Tcrit*err_scifi

print(f'The duration of the Sci-Fi movie should be in between {floor(runtime_scifi_lower)} and {ceil(runtime_scifi_upper)} minutes.')

## Recommendations

**Low Risk:**
* Genre: Horror
* Director: Jordan Peele or John R. Leonetti
* Best release date: Halloween (October) or Easter (April)
* Runtime: 96 to 100 minutes
* Average cost: $7.53 millions

**Medium Risk:**
* Genre: Music
* Director: Damien Chazelle (La La Land)
* Best release date: August
* Runtime: 96 to 113 minutes
* Average cost: $21.93 millions

**High Risk:**
* Genre: Sci-Fi
* Director: Michael Bay (Transformers), Anthony Russo (Marvel), Francis Lawrence (Hunger Games)
* Best release date: March (Spring Break) or June (Summer)
* Runtime: 112 to 120 minutes
* Average cost: $155.73 millions


## Next Steps

For further steps we recommend expanding analysis into several directions:

* Explore data on profits and movie perfomance on streaming services.

* Add budgets spent on marketing. ["Many people get confused when they hear that a movie with a production budget of 100 million grosses 150 million worldwide and loses money. But the brutal fact is that movies get sunk by the massive costs of marketing and distribution all the time."](https://www.studiobinder.com/blog/production-budget/)
